In [ ]:
# default_exp core

# Powerline Detection

> Use Hough Transform to extract powerlines from LiDAR data.

In [ ]:
#hide
from nbdev.showdoc import *
import pathlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import laspy as lp

In [ ]:
#export
import numpy as np
from tqdm import tqdm

In [ ]:
#export
def grid_subsampling(points, voxel_size):
    nb_vox=np.ceil((np.max(points, axis=0) - np.min(points, axis=0))/voxel_size)
    print(nb_vox.shape)
    a = ((points - np.min(points, axis=0)) // voxel_size).astype(int)
    non_empty_voxel_keys, inverse, nb_pts_per_voxel= np.unique(a, axis=0, return_inverse=True, return_counts=True)
    idx_pts_vox_sorted=np.argsort(inverse)
    voxel_grid={}
    grid_barycenter,grid_candidate_center=[],[]
    last_seen=0
    for idx, vox in enumerate(tqdm(non_empty_voxel_keys)):
        per_voxel = nb_pts_per_voxel[idx]
        num_pvs   = last_seen + per_voxel
        idxs      = idx_pts_vox_sorted[last_seen: num_pvs]
        voxel_grid[tuple(vox)] = points[idxs]
        gbc = np.mean(points[idxs], axis=0)
        grid_barycenter.append(gbc)
        #
        temp = voxel_grid[tuple(vox)]-np.mean(voxel_grid[tuple(vox)],axis=0)
        temp = np.linalg.norm(temp, axis=1).argmin()                                             
        grid_candidate_center.append(voxel_grid[tuple(vox)][temp])
        last_seen += per_voxel
    #
    grid_barycenter       = np.array(grid_barycenter)
    grid_candidate_center = np.array(grid_candidate_center)
    return {'barycenter': grid_barycenter, 'candidate': grid_candidate_center}

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
